In [1]:
"""Implementations of algorithms for continuous control."""
import functools
from jaxrl_m.typing import *

import jax
import jax.lax as lax
import jax.numpy as jnp
import numpy as np
import optax
from jaxrl_m.common import TrainState, target_update, nonpytree_field
from jaxrl_m.networks import DeterministicPolicy,Policy, Critic, ensemblize

import flax
import flax.linen as nn
from functools import partial

NUM_CRITICS = 5

class SACAgent(flax.struct.PyTreeNode):
    rng: PRNGKey
    critic: TrainState
    actor : TrainState
    config: dict = nonpytree_field()
    
    @partial(jax.jit,static_argnames=('num_steps',))  
    def update_many_critics(agent,transitions: Batch,idxs:jnp.array,num_steps:int,R2):

        def update_one_critic(critic,idxs,
                            agent,transitions,num_steps):
            
            def one_update(agent,critic,batch: Batch):
                                  
                def critic_loss_fn(critic_params):
                    next_actions = agent.actor(batch['next_observations'])
                    concat_actions = jnp.concatenate([batch["actions"],next_actions])
                    concat_observations = jnp.concatenate([batch["observations"],batch["next_observations"]])
                    
                    concat_q = critic(concat_observations, concat_actions,
                                            True,params=critic_params)
                    q,next_q = jnp.split(concat_q,2,axis=0) ## axis=1 for ensemble
                    
                    target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q
                    target_q = jax.lax.stop_gradient(target_q)
                    
                    critic_loss = ((target_q-q)**2).mean()
                    
                    return critic_loss, {}
        

                
                new_critic, critic_info = critic.apply_loss_fn(loss_fn=critic_loss_fn, has_aux=True)
                
                return agent,new_critic
            
            
            get_batch = lambda transitions,idx : jax.tree_map(lambda x : x[idx],transitions)
                
            agent,new_critic = jax.lax.fori_loop(0, num_steps, 
                        lambda i, args: one_update(*args,get_batch(transitions,idxs[i])),
                        (agent,critic))
            
            return new_critic
        
        
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)
        critic = agent.critic
        
        ###### Reset critic params ######
        
        reset = lambda rng,params : critic.init(rng,
                                                agent.config["observations"], agent.config["actions"],False)["params"]
        no_reset = lambda rng,params: params
        f = lambda  mask,rng,params :lax.cond(mask,reset,no_reset,rng,params)
        mask = jnp.zeros((NUM_CRITICS))
        mask.at[jnp.argmin(R2)].set(1)
        rngs = jax.random.split(agent.rng, NUM_CRITICS)
        critic_params = jax.vmap(f,in_axes=(0,0,0))(mask,rngs,critic.params)
        ###################################
        critic_def = Critic((256,256))
        critics = jax.vmap(TrainState.create,in_axes=(None,0,None))(critic_def,critic_params,optax.adam(learning_rate=3e-4))
        tmp = partial(update_one_critic,agent=agent,transitions=transitions,num_steps=num_steps)
        new_critics = jax.vmap(tmp,in_axes=(0,0))(critics,idxs)
        agent = agent.replace(rng=new_rng,critic=new_critics)
        
    
        return agent,{}
    
    
    @jax.jit
    def update_actor(agent,batch: Batch,R2):
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)
        
        def actor_loss_fn(actor_params,R2):
            
            actions = agent.actor(batch['observations'], params=actor_params)
            
            call_one_critic = lambda observations,actions,params : agent.critic(observations,actions,params=params)
            q = jax.vmap(call_one_critic,in_axes=(None,None,0))(batch['observations'], actions,agent.critic.params)##critic
            q_weights = jax.nn.softmax(R2,axis=0)
            q = jnp.sum(q_weights*q,axis=0)
            q = q*batch['masks']  
            actor_loss = (-q).mean()
            lr_bonus = jnp.exp(jnp.max(R2))/jnp.exp(1)
            actor_loss = lr_bonus*(-q).mean()
            
            return actor_loss, {
                'actor_loss': actor_loss,
              
            }

        loss_fn = partial(actor_loss_fn,R2=R2)
        new_actor, actor_info = agent.actor.apply_loss_fn(loss_fn=loss_fn, has_aux=True)

        return agent.replace(rng=new_rng,actor=new_actor,), {**actor_info}
    
       

    @jax.jit
    def sample_actions(agent, observations: np.ndarray, seed, random=False) -> jnp.ndarray:
        
        actions = agent.actor(observations)
        
        add_noise =  lambda actions,: actions+0.1*jax.random.normal(seed, actions.shape)
        identity = lambda actions : actions
        actions = jax.lax.cond(random, add_noise,identity,actions)
        
        actions = jnp.clip(actions, -1, 1)
        
        return actions
    
 

def create_learner(
                 seed: int,
                 observations: jnp.ndarray,
                 actions: jnp.ndarray,
                 actor_lr: float = 3e-4,
                 critic_lr: float = 3e-4,
                 hidden_dims: Sequence[int] = (256, 256),
                 discount: float = 0.99,
                 
            **kwargs):

        print('Extra kwargs:', kwargs)

        rng = jax.random.PRNGKey(seed)
        rng, actor_key, critic_key = jax.random.split(rng, 3)

        action_dim = actions.shape[-1]
        actor_def = DeterministicPolicy((64,64), action_dim=action_dim,final_fc_init_scale=1.0)

        actor_params = actor_def.init(actor_key, observations)['params']
        actor = TrainState.create(actor_def, actor_params, tx=optax.adam(learning_rate=actor_lr))
       
        critic_def = Critic(hidden_dims)
        critic_keys  = jax.random.split(critic_key, NUM_CRITICS)
        critic_params = jax.vmap(critic_def.init,in_axes=(0,None,None))(critic_keys, observations, actions)['params']
        critics = jax.vmap(TrainState.create,in_axes=(None,0,None))(critic_def,critic_params,optax.adam(learning_rate=critic_lr))

        config = flax.core.FrozenDict(dict(
            discount=discount,
            observations = observations,
            actions = actions,
            
        ))

        return SACAgent(rng,config=config,critic=critics,actor=actor)

In [2]:
import os
from functools import partial
import numpy as np
import jax
import tqdm
from collections import deque
import wandb    
import gymnasium as gym
import copy

from jaxrl_m.utils import *
from jaxrl_m.common import CodeTimer
from jaxrl_m.wandb import setup_wandb, default_wandb_config, get_flag_dict
from jaxrl_m.evaluation import supply_rng, evaluate, flatten, EpisodeMonitor
from jaxrl_m.common import compute_dormant
from jaxrl_m.dataset import ReplayBuffer
from jaxrl_m.rollout import PolicyRollout,rollout_policy,rollout_policy2




NUM_ROLLOUTS = 3
MAX_SIZE = 25

env = 'HalfCheetah'
env_name=env+'-v4'
seed=np.random.choice(1000000)
eval_episodes=10
batch_size = 256
max_steps = int(1e6)
start_steps = 50000                  
log_interval = 5000

wandb_config = default_wandb_config()   
wandb_config.update({
    'project': 'ddpg_on_policy',
    'name': env+"_"+str(seed),
    'hyperparam_dict':{"name":env},
})

env = EpisodeMonitor(gym.make(env_name))
eval_env = EpisodeMonitor(gym.make(env_name))
setup_wandb(**wandb_config)

example_transition = dict(
    observations=env.observation_space.sample(),
    actions=env.action_space.sample(),
    rewards=0.0,
    masks=1.0,
    next_observations=env.observation_space.sample(),
    discounts=1.0,
)   

replay_buffer = ReplayBuffer.create(example_transition, size=int(5e5))
actor_buffer = ReplayBuffer.create(example_transition, size=int(5e3))

agent = create_learner(seed,
                example_transition['observations'][None],
                example_transition['actions'][None],
                max_steps=max_steps,
                )


obs,info = env.reset()    
exploration_rng = jax.random.PRNGKey(0)
unlogged_steps,num_grad_updates,i = 0,0,0
policy_rollouts = deque([], maxlen=MAX_SIZE)
R2 = jnp.ones(NUM_CRITICS)

warmup_steps = 0
while warmup_steps < start_steps:
    
    replay_buffer,_,_,policy_return,undisc_policy_return,num_steps = rollout_policy(agent,env,exploration_rng,
                        replay_buffer,actor_buffer,
                        warmup=False,num_rollouts=NUM_ROLLOUTS)
    warmup_steps += num_steps


random = True
actor_old = agent.actor.params
exploration_key = jax.random.PRNGKey(0)
first = True
with tqdm.tqdm(total=max_steps) as pbar:
    
    while i < max_steps:

        ### Rollout policy ###
        num_rollouts =  2 if random else NUM_ROLLOUTS

        actor_buffer = ReplayBuffer.create(example_transition, size=int(NUM_ROLLOUTS*1000))
        replay_buffer,actor_buffer,policy_rollout,policy_return,undisc_policy_return,num_steps = rollout_policy(
                                                                agent,env,exploration_rng,
                                                                replay_buffer,actor_buffer,warmup=False,
                                                                num_rollouts=num_rollouts,random=random,
                                                                )
        i+=num_steps
        unlogged_steps += num_steps
        pbar.update(num_steps)
        
        if not random : 
            
            policy_rollouts.append(policy_rollout)
            
            ### Update critics ###
            transitions = replay_buffer.get_all()
            tmp = partial(jax.random.choice,a=replay_buffer.size, shape=(10000,256), replace=True)
            idxs = jax.vmap(tmp)(jax.random.split(exploration_key, NUM_CRITICS))
            agent, critic_update_info = agent.update_many_critics(transitions,idxs,10000,R2)

            ### Update critic weights ##                    
            flattened_rollouts = flatten_rollouts(policy_rollouts)
            R2,bias,_,_ = train_evaluation(agent,policy_rollout.policy_return,flattened_rollouts)
         
            ### Update actor ###        
            actor_batch = actor_buffer.get_all() 
            agent, actor_update_info = agent.update_actor(actor_batch,R2.reshape(-1,1))   
            
            ### Divers ###
            percent_dormant = compute_dormant(agent,policy_rollout.observations,0.2)
            action_distance = measure_action_distance(agent,agent.actor.params,actor_old,policy_rollout.observations)
            exploration_rng, key = jax.random.split(exploration_rng)
            actor_old = agent.actor.params                    
            num_grad_updates += 1 

        if unlogged_steps >= log_interval:

            ### Log stuff to w&b ### 
            update_info = {**critic_update_info, **actor_update_info}
            R2_train_info = {'max': jnp.max(R2),'bias': bias[jnp.argmax(R2)],
                                "histogram": wandb.Histogram(jnp.clip(R2,a_min=-1,a_max=1)),
                                }
            R2_metrics = {f'R2/{k}': v for k, v in R2_train_info.items()}
            exploration_metrics = {f'exploration/disc_return': policy_return}
            train_metrics = {f'training/{k}': v for k, v in update_info.items()}
            
            wandb.log(train_metrics, step=int(i),commit=False)
            wandb.log({"undisc_return":undisc_policy_return,"dormant":percent_dormant,"action_distance":action_distance},step=int(i),commit=False)
            wandb.log(R2_metrics, step=int(i),commit=False)
            wandb.log(exploration_metrics, step=int(i),commit=True)
            
            unlogged_steps = 0

        random = not random
    

2024-01-24 22:34:55.386320: W external/xla/xla/service/gpu/nvptx_compiler.cc:679] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.103). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin


Extra kwargs: {'max_steps': 1000000}


KeyboardInterrupt: 

: 

In [4]:
import numpy as np

disc = 0.99
rewards = [disc**i for i in range(1000)]
rewards = np.cumsum(rewards)*(1-disc)
#np.argmax(rewards>0.995)
rewards[128]


0.7265108489777838